# Brandon Fong's Research Journal

Journal for necessary skills, machines, code, formulas, and more to build projects.

---


# Physical Qubit State Visualizer

**Objective**: create a motorized Bloch Sphere with an interface to update a single qubit's pure state

A qubit's state is defined by the formula $$\ket{\psi}=\alpha\ket{0}+\beta\ket{1}$$ where $\alpha$ and $\beta$ are both imaginary numbers. For $\ket{\psi}$ to be a pure state, $|\alpha|^2+|\beta|^2$ must equal $1$.


## 1. Qubit Class

**Objective**: implement a qubit using NumPy and Python classes


In [ ]:
import numpy as np
import numpy.typing as npt

class Qubit:
    def __init__(self, alpha:complex=1+0j, beta:complex=0+0j) -> None:
        if abs(abs(alpha)**2 + abs(beta)**2 - 1) > 1e-5:
            raise ValueError("The Euclidean norm of the qubit state must be 1")
        self.state: npt.NDArray  = np.array([alpha, beta])

    def __repr__(self) -> str:
        return f"[{self.state[0]:.2f}, {self.state[1]:.2f}]"
    
    def get_state(self):
        return self.state

    def set_state(self, alpha:complex, beta:complex) -> None:
        self.state = np.array([alpha, beta])

    def x(self) -> None:
        x = np.array([[0, 1], [1, 0]])
        self.state = self.state @ x

    def z(self) -> None:
        z = np.array([[1, 0], [0, -1]])
        self.state = self.state @ z

    def h(self) -> None:
        h = 1/np.sqrt(2) * np.array([[1, 1], [1, -1]])
        self.state = self.state @ h

[0.71+0.00j, 0.71+0.00j]
[1.00+0.00j, 0.00+0.00j]
[0.00+0.00j, 1.00+0.00j]


# 2. Creating Simple Interface with Typescript + Next.js


The interface will be a simple, one-page website using the Next.js framework.


## 2.1 Implementing an API

I will use FastAPI as my Python backend and Next.js' built-in API routes as my frontend.

I installed FastAPI in my Python .venv and initialized a backend using the following code.


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/api/data")
async def root():
    return {"message": "Hello World"}

Before communicating, a middleware must be added to allow the Next.js origin to access the API:


In [ ]:
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http:/localhost:3000"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

To allow Next.js to communicate with FastAPI through HTTP requests, I will use the Next.js fetch API.


In [ ]:
// My FastAPI application is running on port 8000
const response = await fetch("http://localhost:8000/api/data");
if (!response.ok) {
  throw new Error("Network response was not ok");
}
const result = await response.json();


The python data shows up correctly on the Next.js website!


## 2.2 GET Requests

Because JSON and TypeScript don't have a built-in imaginary number system, the qubit's amplitudes must be sent over through four distinct variables:

- `alpha_real`
- `alpha_imag`
- `beta_real`
- `beta_imag`


In [ ]:
# Initialize Qubit
q = Qubit()

# Separate the state into its components
alpha, beta = q.get_state()

@app.get("/api/data")
async def root():
    return {"alpha_real": alpha.real,
            "alpha_imag": alpha.imag,
            "beta_real": beta.real,
            "beta_imag": beta.imag}

## 2.3 Formatting Math

Before implementing the qubit gate and state functionality, I must first format the fetched data into readable Dirac notation. This can be achieved through MathJax using the package `better-react-mathjax`.


In [ ]:
// Configuration
const mathJaxConfig = {
  tex: {
    inlineMath: [
      ["$", "$"],
      ["\\(", "\\)"],
    ],
    displayMath: [
      ["$$", "$$"],
      ["\\[", "\\]"],
    ],
  },
};


Here's an example


In [ ]:
<MathJaxContext config={mathJaxConfig}>
  {loading && <span>Loading...</span>}
  {error && <span>Error: {error}</span>}
  {amps && <MathJax>{`$$\\ket{\\psi}=n/a$$`}</MathJax>}
</MathJaxContext>;
